In [1]:
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import theano
import theano.tensor as T
import pickle
import time
from tqdm import tqdm

%matplotlib inline

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5110)


In [2]:
filters = ['Original', 'Gingham', 'Helena', 'Kelvin', 'Ludwig', 'Moon', 'Slumber', 'Ginza', 'Juno', 'Lo-Fi', 'Maven', 'Toaster']

In [65]:
import lasagne
from lasagne.layers import InputLayer
from lasagne.layers import Conv2DLayer as ConvLayer
from lasagne.layers import BatchNormLayer
from lasagne.layers import Pool2DLayer as PoolLayer
from lasagne.layers import NonlinearityLayer
from lasagne.layers import ElemwiseSumLayer
from lasagne.layers import DenseLayer
from lasagne.nonlinearities import rectify, softmax

def build_simple_block(incoming_layer, names,
                       num_filters, filter_size, stride, pad,
                       use_bias=False, nonlin=rectify):
    names = list(names)
    net = []
    net.append((
            names[0],
            ConvLayer(incoming_layer, num_filters, filter_size, stride, pad,
                      flip_filters=False, nonlinearity=None) if use_bias
            else ConvLayer(incoming_layer, num_filters, filter_size, stride, pad, b=None,
                           flip_filters=False, nonlinearity=None)
        ))

    net.append((
            names[1],
            BatchNormLayer(net[-1][1])
        ))
    if nonlin is not None:
        net.append((
            names[2],
            NonlinearityLayer(net[-1][1], nonlinearity=nonlin)
        ))

    return dict(net), net[-1][0]


def build_residual_block(incoming_layer, ratio_n_filter=1.0, ratio_size=1.0, has_left_branch=False,
                         upscale_factor=4, ix=''):
    simple_block_name_pattern = ['res%s_branch%i%s', 'bn%s_branch%i%s', 'res%s_branch%i%s_relu']

    net = {}

    # right branch
    net_tmp, last_layer_name = build_simple_block(
        incoming_layer, map(lambda s: s % (ix, 2, 'a'), simple_block_name_pattern),
        int(lasagne.layers.get_output_shape(incoming_layer)[1]*ratio_n_filter), 1, int(1.0/ratio_size), 0)
    net.update(net_tmp)

    net_tmp, last_layer_name = build_simple_block(
        net[last_layer_name], map(lambda s: s % (ix, 2, 'b'), simple_block_name_pattern),
        lasagne.layers.get_output_shape(net[last_layer_name])[1], 3, 1, 1)
    net.update(net_tmp)

    net_tmp, last_layer_name = build_simple_block(
        net[last_layer_name], map(lambda s: s % (ix, 2, 'c'), simple_block_name_pattern),
        lasagne.layers.get_output_shape(net[last_layer_name])[1]*upscale_factor, 1, 1, 0,
        nonlin=None)
    net.update(net_tmp)

    right_tail = net[last_layer_name]
    left_tail = incoming_layer

    # left branch
    if has_left_branch:
        net_tmp, last_layer_name = build_simple_block(
            incoming_layer, map(lambda s: s % (ix, 1, ''), simple_block_name_pattern),
            int(lasagne.layers.get_output_shape(incoming_layer)[1]*4*ratio_n_filter), 1, int(1.0/ratio_size), 0,
            nonlin=None)
        net.update(net_tmp)
        left_tail = net[last_layer_name]

    net['res%s' % ix] = ElemwiseSumLayer([left_tail, right_tail], coeffs=1)
    net['res%s_relu' % ix] = NonlinearityLayer(net['res%s' % ix], nonlinearity=rectify)

    return net, 'res%s_relu' % ix


def build_model():
    net = {}
    net['input'] = InputLayer((None, 3, 224, 224))
    sub_net, parent_layer_name = build_simple_block(
        net['input'], ['conv1', 'bn_conv1', 'conv1_relu'],
        64, 7, 2, 3, use_bias=True)
    net.update(sub_net)
    net['pool1'] = PoolLayer(net[parent_layer_name], pool_size=3, stride=2, pad=0, mode='max', ignore_border=False)
    block_size = list('abc')
    parent_layer_name = 'pool1'
    for c in block_size:
        if c == 'a':
            sub_net, parent_layer_name = build_residual_block(net[parent_layer_name], 1, 1, True, 4, ix='2%s' % c)
        else:
            sub_net, parent_layer_name = build_residual_block(net[parent_layer_name], 1.0/4, 1, False, 4, ix='2%s' % c)
        net.update(sub_net)

    block_size = list('abcd')
    for c in block_size:
        if c == 'a':
            sub_net, parent_layer_name = build_residual_block(
                net[parent_layer_name], 1.0/2, 1.0/2, True, 4, ix='3%s' % c)
        else:
            sub_net, parent_layer_name = build_residual_block(net[parent_layer_name], 1.0/4, 1, False, 4, ix='3%s' % c)
        net.update(sub_net)

    block_size = list('abcdef')
    for c in block_size:
        if c == 'a':
            sub_net, parent_layer_name = build_residual_block(
                net[parent_layer_name], 1.0/2, 1.0/2, True, 4, ix='4%s' % c)
        else:
            sub_net, parent_layer_name = build_residual_block(net[parent_layer_name], 1.0/4, 1, False, 4, ix='4%s' % c)
        net.update(sub_net)

    block_size = list('abc')
    for c in block_size:
        if c == 'a':
            sub_net, parent_layer_name = build_residual_block(
                net[parent_layer_name], 1.0/2, 1.0/2, True, 4, ix='5%s' % c)
        else:
            sub_net, parent_layer_name = build_residual_block(net[parent_layer_name], 1.0/4, 1, False, 4, ix='5%s' % c)
        net.update(sub_net)
    net['pool5'] = PoolLayer(net[parent_layer_name], pool_size=7, stride=1, pad=0,
                             mode='average_exc_pad', ignore_border=False)
    net['fc1000'] = DenseLayer(net['pool5'], num_units=1000, nonlinearity=None)
    net['prob'] = NonlinearityLayer(net['fc1000'], nonlinearity=softmax)

    return net

In [66]:
weights = pickle.load(open('resnet50.pkl', 'rb'), encoding='latin1')
net = build_model()
lasagne.layers.set_all_param_values(net['prob'], weights['values'])

In [5]:
inpt = T.tensor4()
prep_fn = theano.function([inpt], lasagne.layers.get_output(net['pool5'], inpt, deterministic=True))

/home/ubuntu/env/src/lasagne/lasagne/layers/pool.py:266: UserWarning: DEPRECATION: the 'ds' parameter is not going to exist anymore as it is going to be replaced by the parameter 'ws'.
  mode=self.mode,
/home/ubuntu/env/src/lasagne/lasagne/layers/pool.py:266: UserWarning: DEPRECATION: the 'st' parameter is not going to exist anymore as it is going to be replaced by the parameter 'stride'.
  mode=self.mode,
/home/ubuntu/env/src/lasagne/lasagne/layers/pool.py:266: UserWarning: DEPRECATION: the 'padding' parameter is not going to exist anymore as it is going to be replaced by the parameter 'pad'.
  mode=self.mode,


In [58]:
nnet = {}
nnet['input'] = InputLayer([None, 2048])
nnet['dense'] = lasagne.layers.DenseLayer(nnet['input'], num_units=1024)
nnet['prob'] = lasagne.layers.DenseLayer(nnet['dense'], num_units=len(filters),
                                         nonlinearity=lasagne.nonlinearities.softmax)

In [59]:
target = T.ivector()
inpt = T.matrix()

pred = lasagne.layers.get_output(nnet['prob'], inpt, deterministic=False)
pred_det = lasagne.layers.get_output(nnet['prob'], inpt, deterministic=True)

loss = lasagne.objectives.categorical_crossentropy(pred, target).mean()
loss += 1e-5 * lasagne.regularization.regularize_network_params(nnet['prob'], lasagne.regularization.l2)
loss_det = lasagne.objectives.categorical_crossentropy(pred_det, target).mean()

acc = lasagne.objectives.categorical_accuracy(pred_det, target).mean()

params = []
for k, v in nnet.items():
    params.extend(v.get_params())

updates = lasagne.updates.adam(loss, params)

In [60]:
train = theano.function([inpt, target], [loss], updates=updates)
predict_prob = theano.function([inpt], [pred_det])
pred_acc = theano.function([inpt, target], [acc])

In [9]:
H, W = 224, 224

In [10]:
import skimage.color

def preprocess(im):
    im = skimage.color.gray2rgb(im)
    im = np.swapaxes(np.swapaxes(im, 1, 2), 0, 1)
    im = im[::-1, :, :]
    return (im - weights['mean_image']).astype(np.float32)

def crop_center(img, H=112, W=112):
    h,w = img.shape[:2]
    hc, wc = h // 2, w // 2
    return img[hc - H: hc + H, wc - W : wc + W]

def augment(x):
    return [x[:H, :W], x[-H:, -W:], x[:H, -W:], x[-H:, :W], crop_center(x)]

In [11]:
def myprocess(im):
    return prep_fn(list(map(preprocess, augment(im))))

In [28]:
X_train, X_test, y_train, y_test = [], [], [], []
for fi, f in enumerate(filters):
    for i in tqdm(range(100, 997)):
        try:
            nx = myprocess(plt.imread(f + '/' + str(i) + '.jpg'))
            if i < 200:
                X_test.extend(nx)
                y_test.extend([fi] * len(nx))
            else:
                X_train.extend(nx)
                y_train.extend([fi] * len(nx))
        except:
            pass
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

100%|██████████| 897/897 [01:59<00:00,  7.55it/s]


In [7]:
#pickle.dump((X_train, X_test, y_train, y_test), open('traintestdata.pkl', 'wb'))
X_train, X_test, y_train, y_test  = pickle.load(open('traintestdata.pkl', 'rb'))

In [30]:
def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert len(inputs) == len(targets)
    indices = np.arange(len(inputs))
    if shuffle:
        np.random.shuffle(indices)
    start_idx = 0
    for start_idx in range(0, len(inputs), batchsize):
        excerpt = indices[start_idx: min(len(inputs), start_idx + batchsize)]
        yield inputs[excerpt], targets[excerpt]

In [50]:
epoch = 0
err_log = []
val_log = []

In [61]:
for i in range(100):
    err = 0
    batches = 0
    t = time.time()
    for X_batch, y_batch in iterate_minibatches(X_train, y_train, 1000, True):
        err += train(X_batch, y_batch)[0]
        batches += 1
    err_log.append(err / batches)
    
    err = 0
    batches = 0
    for X_batch, y_batch in iterate_minibatches(X_test, y_test, 500, True):
        err += pred_acc(X_batch, y_batch)[0]
        batches += 1
    val_log.append(err / batches)
    print('epoch: {}\ttook {}\ttrain: {}\tval acc: {}'.format(epoch, time.time() - t, err_log[-1], val_log[-1]))
    if val_log[-1] > 0.801:
        break
    epoch += 1

epoch: 113	took 0.8366317749023438	train: 1.7569742525617282	val acc: 0.6993333333333333
epoch: 114	took 0.7890450954437256	train: 0.8205591601630052	val acc: 0.7475
epoch: 115	took 0.7291712760925293	train: 0.6430496374766032	val acc: 0.761
epoch: 116	took 0.7265677452087402	train: 0.5465609511981407	val acc: 0.7815
epoch: 117	took 0.7236247062683105	train: 0.46947560645639896	val acc: 0.7838333333333334
epoch: 118	took 0.7228283882141113	train: 0.4130351065347592	val acc: 0.7915
epoch: 119	took 0.7156577110290527	train: 0.3669953513890505	val acc: 0.7906666666666667
epoch: 120	took 0.7154340744018555	train: 0.32051418349146843	val acc: 0.7945000000000001
epoch: 121	took 0.7226624488830566	train: 0.2844625298554699	val acc: 0.7933333333333333
epoch: 122	took 0.7160594463348389	train: 0.24436376926799616	val acc: 0.8016666666666666


In [67]:
nnet['dense'].input_layer = net['pool5']

In [68]:
data = {'values':lasagne.layers.get_all_param_values(nnet['prob']),
       'mean_image': weights['mean_image']}

In [69]:
pickle.dump(data, open('resnet50_final.pkl', 'wb'))

In [95]:
import theano
import theano.tensor as T
import lasagne
from lasagne.layers import InputLayer
from lasagne.layers import Conv2DLayer as ConvLayer
from lasagne.layers import BatchNormLayer
from lasagne.layers import Pool2DLayer as PoolLayer
from lasagne.layers import NonlinearityLayer
from lasagne.layers import ElemwiseSumLayer
from lasagne.layers import DenseLayer
from lasagne.nonlinearities import rectify, softmax
import skimage.color

def build_simple_block(incoming_layer, names,
                       num_filters, filter_size, stride, pad,
                       use_bias=False, nonlin=rectify):
    names = list(names)
    net = []
    net.append((
            names[0],
            ConvLayer(incoming_layer, num_filters, filter_size, stride, pad,
                      flip_filters=False, nonlinearity=None) if use_bias
            else ConvLayer(incoming_layer, num_filters, filter_size, stride, pad, b=None,
                           flip_filters=False, nonlinearity=None)
        ))

    net.append((
            names[1],
            BatchNormLayer(net[-1][1])
        ))
    if nonlin is not None:
        net.append((
            names[2],
            NonlinearityLayer(net[-1][1], nonlinearity=nonlin)
        ))

    return dict(net), net[-1][0]


def build_residual_block(incoming_layer, ratio_n_filter=1.0, ratio_size=1.0, has_left_branch=False,
                         upscale_factor=4, ix=''):
    simple_block_name_pattern = ['res%s_branch%i%s', 'bn%s_branch%i%s', 'res%s_branch%i%s_relu']

    net = {}

    # right branch
    net_tmp, last_layer_name = build_simple_block(
        incoming_layer, map(lambda s: s % (ix, 2, 'a'), simple_block_name_pattern),
        int(lasagne.layers.get_output_shape(incoming_layer)[1]*ratio_n_filter), 1, int(1.0/ratio_size), 0)
    net.update(net_tmp)

    net_tmp, last_layer_name = build_simple_block(
        net[last_layer_name], map(lambda s: s % (ix, 2, 'b'), simple_block_name_pattern),
        lasagne.layers.get_output_shape(net[last_layer_name])[1], 3, 1, 1)
    net.update(net_tmp)

    net_tmp, last_layer_name = build_simple_block(
        net[last_layer_name], map(lambda s: s % (ix, 2, 'c'), simple_block_name_pattern),
        lasagne.layers.get_output_shape(net[last_layer_name])[1]*upscale_factor, 1, 1, 0,
        nonlin=None)
    net.update(net_tmp)

    right_tail = net[last_layer_name]
    left_tail = incoming_layer

    # left branch
    if has_left_branch:
        net_tmp, last_layer_name = build_simple_block(
            incoming_layer, map(lambda s: s % (ix, 1, ''), simple_block_name_pattern),
            int(lasagne.layers.get_output_shape(incoming_layer)[1]*4*ratio_n_filter), 1, int(1.0/ratio_size), 0,
            nonlin=None)
        net.update(net_tmp)
        left_tail = net[last_layer_name]

    net['res%s' % ix] = ElemwiseSumLayer([left_tail, right_tail], coeffs=1)
    net['res%s_relu' % ix] = NonlinearityLayer(net['res%s' % ix], nonlinearity=rectify)

    return net, 'res%s_relu' % ix


def build_model_my_resnet():
    net = {}
    net['input'] = InputLayer((None, 3, 224, 224))
    sub_net, parent_layer_name = build_simple_block(
        net['input'], ['conv1', 'bn_conv1', 'conv1_relu'],
        64, 7, 2, 3, use_bias=True)
    net.update(sub_net)
    net['pool1'] = PoolLayer(net[parent_layer_name], pool_size=3, stride=2, pad=0, mode='max', ignore_border=False)
    block_size = list('abc')
    parent_layer_name = 'pool1'
    for c in block_size:
        if c == 'a':
            sub_net, parent_layer_name = build_residual_block(net[parent_layer_name], 1, 1, True, 4, ix='2%s' % c)
        else:
            sub_net, parent_layer_name = build_residual_block(net[parent_layer_name], 1.0/4, 1, False, 4, ix='2%s' % c)
        net.update(sub_net)

    block_size = list('abcd')
    for c in block_size:
        if c == 'a':
            sub_net, parent_layer_name = build_residual_block(
                net[parent_layer_name], 1.0/2, 1.0/2, True, 4, ix='3%s' % c)
        else:
            sub_net, parent_layer_name = build_residual_block(net[parent_layer_name], 1.0/4, 1, False, 4, ix='3%s' % c)
        net.update(sub_net)

    block_size = list('abcdef')
    for c in block_size:
        if c == 'a':
            sub_net, parent_layer_name = build_residual_block(
                net[parent_layer_name], 1.0/2, 1.0/2, True, 4, ix='4%s' % c)
        else:
            sub_net, parent_layer_name = build_residual_block(net[parent_layer_name], 1.0/4, 1, False, 4, ix='4%s' % c)
        net.update(sub_net)

    block_size = list('abc')
    for c in block_size:
        if c == 'a':
            sub_net, parent_layer_name = build_residual_block(
                net[parent_layer_name], 1.0/2, 1.0/2, True, 4, ix='5%s' % c)
        else:
            sub_net, parent_layer_name = build_residual_block(net[parent_layer_name], 1.0/4, 1, False, 4, ix='5%s' % c)
        net.update(sub_net)
    net['pool5'] = PoolLayer(net[parent_layer_name], pool_size=7, stride=1, pad=0,
                             mode='average_exc_pad', ignore_border=False)
    net['dense'] = lasagne.layers.DenseLayer(net['pool5'], num_units=1024)
    net['prob'] = lasagne.layers.DenseLayer(net['dense'], num_units=12,
                                         nonlinearity=lasagne.nonlinearities.softmax)

    return net


class ResNetModel:
    def preprocess(self, im):
        im = skimage.color.gray2rgb(im)
        im = np.swapaxes(np.swapaxes(im, 1, 2), 0, 1)
        im = im[::-1, :, :]
        return (im - self._mean_image).astype(np.float32)
    
    def crop_center(self, img, H=112, W=112):
        h,w = img.shape[:2]
        hc, wc = h // 2, w // 2
        return img[hc - H: hc + H, wc - W : wc + W]
    
    def augment(self, x, H=224, W=224):
        return [x[:H, :W], x[-H:, -W:], x[:H, -W:], x[-H:, :W], self.crop_center(x)]
    
    def load(self):
        weights = pickle.load(open('resnet50_final.pkl', 'rb'))
        self._layers = build_model_my_resnet()
        lasagne.layers.set_all_param_values(self._layers['prob'], weights['values'])
        self._mean_image = weights['mean_image']
        inpt = T.tensor4()
        self._function = theano.function([inpt], lasagne.layers.get_output(self._layers['prob']
                                                                           , inpt, deterministic=True))
    def predict(self, X):
        X = sum([[self.preprocess(j) for j in self.augment(i)] for i in X], [])
        res = self._function(X)
        return res.reshape([-1,5,12]).mean(axis=1)
    
    def unload(self):
        del self._layers
        del self._function

In [96]:
m = ResNetModel()
m.load()

/home/ubuntu/env/src/lasagne/lasagne/layers/pool.py:266: UserWarning: DEPRECATION: the 'ds' parameter is not going to exist anymore as it is going to be replaced by the parameter 'ws'.
  mode=self.mode,
/home/ubuntu/env/src/lasagne/lasagne/layers/pool.py:266: UserWarning: DEPRECATION: the 'st' parameter is not going to exist anymore as it is going to be replaced by the parameter 'stride'.
  mode=self.mode,
/home/ubuntu/env/src/lasagne/lasagne/layers/pool.py:266: UserWarning: DEPRECATION: the 'padding' parameter is not going to exist anymore as it is going to be replaced by the parameter 'pad'.
  mode=self.mode,


In [105]:
m.predict([im]).argmax()

(5, 12)


7